In [ ]:
# === Install library yang dibutuhkan (hanya sekali di Colab) ===
!pip install --quiet gspread gspread-dataframe oauth2client pandas requests


In [ ]:
import pandas as pd
import requests
import gspread
import re
from oauth2client.service_account import ServiceAccountCredentials

# Konstanta
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/12taGrqZYfb9F91eIo9q2oVKsnpxAIqFbVFOya1LSxyg/edit"
OLLAMA_URL = " https://salad-female-some-populations.trycloudflare.com"
OLLAMA_MODEL = "llama3"

# Setup koneksi Google Sheets
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name("/content/credentials.json", scope)
client = gspread.authorize(creds)

# Buka spreadsheet dan worksheets
sheet = client.open_by_url(SPREADSHEET_URL)
wawancara_ws = sheet.worksheet("Hasil Wawancara")
rubrik_ws = sheet.worksheet("Penilaian Rubrik")


In [ ]:
# Ambil seluruh isi sheet wawancara
df_wawancara = pd.DataFrame(wawancara_ws.get_all_values())
df_wawancara.columns = df_wawancara.iloc[0]
df_wawancara = df_wawancara[1:]

# Ambil kolom F–W (index 5 s.d. 22)
df_deskripsi = df_wawancara.iloc[:, 5:23]

# Gabungkan menjadi satu string per UMKM
def gabung_deskripsi(row):
    return ' '.join([str(x) for x in row if str(x).strip()])

# Ambil deskripsi UMKM pertama (jika hanya 1 yang dianalisis)
deskripsi_umkm = gabung_deskripsi(df_deskripsi.iloc[0])


In [ ]:
import re
import requests

def analyze_skor(kriteria, rubrik, deskripsi):
    prompt = f"""
Deskripsi UMKM:
\"{deskripsi}\"

Kriteria Penilaian:
\"{kriteria}\"

Rubrik Penilaian:
\"{rubrik}\"

Tentukan skor digitalisasi untuk kriteria tersebut:
- 0: Belum ada digitalisasi
- 1: Digitalisasi awal dan terbatas
- 2: Terintegrasi sebagian
- 3: Transformasi menyeluruh

Kembalikan hanya 1 angka: 0, 1, 2, atau 3. Jangan berikan penjelasan tambahan.
"""
    try:
        res = requests.post(
            f"{OLLAMA_URL}/api/generate",
            json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False},
            timeout=60
        )
        output = res.text.strip()
        match = re.search(r'\b[0-3]\b', output)
        if match:
            return match.group(0)
        else:
            print(f"⚠️ Jawaban tidak valid untuk kriteria '{kriteria}': {output} → default = 0")
            return "0"
    except requests.exceptions.ReadTimeout:
        print(f"❌ Timeout saat analisis kriteria '{kriteria}' → default = 0")
        return "0"
    except Exception as e:
        print(f"❌ Error saat analisis kriteria '{kriteria}': {e} → default = 0")
        return "0"


In [ ]:
# Ambil isi sheet rubrik
rubrik_data = rubrik_ws.get_all_values()
header, rows = rubrik_data[0], rubrik_data[1:]

# Loop per baris rubrik
for i, row in enumerate(rows):
    kriteria = row[0]
    rubrik = row[1]
    skor = analyze_skor(kriteria, rubrik, deskripsi_umkm)
    rubrik_ws.update_acell(f"C{i+2}", skor)

print("✅ Semua skor berhasil ditulis ke kolom C Sheet 'Penilaian Rubrik'")


❌ Timeout saat analisis kriteria 'Penggunaan teknologi digital' → default = 0


KeyboardInterrupt: 

In [ ]:
# -*- coding: utf-8 -*-
!pip install --quiet gspread gspread-dataframe oauth2client pandas requests

import pandas as pd
import requests
import gspread
import re
from oauth2client.service_account import ServiceAccountCredentials

# === KONSTANTA ===
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/12taGrqZYfb9F91eIo9q2oVKsnpxAIqFbVFOya1LSxyg/edit"
OLLAMA_URL = " https://salad-female-some-populations.trycloudflare.com"
OLLAMA_MODEL = "llama3"

# === AUTENTIKASI GSPREAD ===
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/credentials.json", scope)
client = gspread.authorize(creds)

# === AKSES SHEETS ===
sheet = client.open_by_url(SPREADSHEET_URL)
wawancara_ws = sheet.worksheet("Hasil Wawancara")
rubrik_ws = sheet.worksheet("Penilaian Rubrik")

# === AMBIL DATA ===
df_wawancara = pd.DataFrame(wawancara_ws.get_all_values())
df_wawancara.columns = df_wawancara.iloc[0]
df_wawancara = df_wawancara[1:]

rubrik_data = rubrik_ws.get_all_values()
rubrik_header = rubrik_data[0]
rubrik_rows = rubrik_data[1:]

# === GABUNG JAWABAN DESKRIPTIF UMKM ===
def gabung_deskripsi(row):
    return ' '.join([str(x).strip() for x in row if str(x).strip()])

# === ANALISIS DENGAN OLLAMA ===
def analyze_skor(kriteria, rubrik, deskripsi):
    prompt = f"""
Deskripsi UMKM:
\"{deskripsi}\"

Kriteria Penilaian:
\"{kriteria}\"

Rubrik Penilaian:
\"{rubrik}\"

Tentukan skor digitalisasi untuk kriteria tersebut:
- 0: Belum ada digitalisasi
- 1: Digitalisasi awal dan terbatas
- 2: Terintegrasi sebagian
- 3: Transformasi menyeluruh

Kembalikan hanya 1 angka: 0, 1, 2, atau 3. Jangan berikan penjelasan tambahan.
"""
    try:
        res = requests.post(
            f"{OLLAMA_URL}/api/generate",
            json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False},
            timeout=240
        )
        output = res.text.strip()
        match = re.search(r'\b[0-3]\b', output)
        return match.group(0) if match else "0"
    except Exception as e:
        print(f"❌ Error saat analisis kriteria '{kriteria}': {e} → default = 0")
        return "0"

# === PROSES SETIAP PT YANG VALID ===
jumlah_umkm = 0

for idx, row in df_wawancara.iterrows():
    nama_industri = row['Nama Industri']
    if not nama_industri.strip():
        print(f"⏹️ Baris {idx+2} kosong → proses berhenti.")
        break  # Stop jika baris berikutnya kosong

    kolom_output = chr(ord('C') + jumlah_umkm)  # C, D, E, ...
    deskripsi = gabung_deskripsi(row.iloc[5:23])  # Kolom F–W

    print(f"\n🔍 Memproses '{nama_industri}' → Kolom {kolom_output}")

    for i, rubrik_row in enumerate(rubrik_rows):
        kriteria = rubrik_row[0]
        rubrik = rubrik_row[1]
        skor = analyze_skor(kriteria, rubrik, deskripsi)
        rubrik_ws.update_acell(f"{kolom_output}{i + 2}", skor)

    jumlah_umkm += 1  # Naik ke kolom berikutnya

print("\n✅ Semua skor berhasil ditulis ke Sheet 'Penilaian Rubrik'")



🔍 Memproses 'PT CIFA Indonesia' → Kolom C
❌ Error saat analisis kriteria 'Strategi Digital dan Bisnis Inklusif': ('Connection broken: IncompleteRead(0 bytes read, 55 more expected)', IncompleteRead(0 bytes read, 55 more expected)) → default = 0
❌ Error saat analisis kriteria 'Eksplorasi dan Eksploitasi Teknologi': ('Connection broken: IncompleteRead(0 bytes read, 55 more expected)', IncompleteRead(0 bytes read, 55 more expected)) → default = 0
❌ Error saat analisis kriteria 'Kolaborasi bisnis dengan organisasi lain': ('Connection broken: IncompleteRead(0 bytes read, 55 more expected)', IncompleteRead(0 bytes read, 55 more expected)) → default = 0
❌ Error saat analisis kriteria 'Inovasi produk atau jasa berbasis teknologi': ('Connection broken: IncompleteRead(0 bytes read, 55 more expected)', IncompleteRead(0 bytes read, 55 more expected)) → default = 0
❌ Error saat analisis kriteria 'Pengembangan solusi teknologi responsif': ('Connection broken: IncompleteRead(0 bytes read, 55 more ex